In [1]:
from keras.applications.densenet import DenseNet201

conv_base = DenseNet201(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

Using TensorFlow backend.


In [2]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 75, 75, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [3]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        directory="Data/Train",
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = test_datagen.flow_from_directory(
        directory="Data/Valid",
        target_size=(150, 150),
        batch_size=64,
        class_mode='binary')


Found 903 images belonging to 12 classes.
Found 360 images belonging to 12 classes.


In [5]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'pool4_pool':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [6]:
from keras import optimizers

model.compile(loss='cosine_proximity',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [7]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=150,
      epochs=75,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/75
150/150 [==============================] - 162s 1s/step - loss: -0.9141 - acc: 0.0899 - val_loss: -0.9162 - val_acc: 0.0751
Epoch 2/75
150/150 [==============================] - 106s 707ms/step - loss: -0.9149 - acc: 0.0865 - val_loss: -0.9156 - val_acc: 0.0672
Epoch 3/75
150/150 [==============================] - 106s 704ms/step - loss: -0.9157 - acc: 0.0835 - val_loss: -0.9182 - val_acc: 0.0714
Epoch 4/75
150/150 [==============================] - 106s 706ms/step - loss: -0.9174 - acc: 0.0850 - val_loss: -0.9159 - val_acc: 0.0788
Epoch 5/75
150/150 [==============================] - 106s 710ms/step - loss: -0.9166 - acc: 0.0812 - val_loss: -0.9159 - val_acc: 0.0831
Epoch 6/75
150/150 [==============================] - 107s 710ms/step - loss: -0.9183 - acc: 0.0814 - val_loss: -0.9182 - val_acc: 0.0794
Epoch 7/75
150/150 [==============================] - 107s 711ms/step - loss: -0.9174 - acc: 0.0846 - val_loss: -0.9179 - val_acc: 0.0801
Epoch 8/75
150/150 [=================

In [8]:
import numpy as np
model.save('DenseMod2.h5')
np.save('DenseMod2', history.history)

In [9]:
import numpy as np
history_dict = np.load('DenseMod2.npy')
model.load_weights('DenseMod2.h5')


In [10]:
acc = history_dict.item().get('acc')
val_acc = history_dict.item().get('val_acc')
loss = history_dict.item().get('loss')
val_loss = history_dict.item().get('val_loss')

In [11]:
import matplotlib.pyplot as plt


epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [12]:
test_generator = test_datagen.flow_from_directory(
        directory="Data/Test",
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 554 images belonging to 12 classes.


In [13]:

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc,'test loss',test_loss)

test acc: 0.08551308019480235 test loss -0.9094567362451458


In [14]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 75, 75, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu